## Spatial Augmentations & Updating Annotations

In [ ]:
import os
import cv2
import pandas as pd
import albumentations as A
from tqdm import tqdm

# === Paths ===
image_dir = r"D:\Suvan's Projects\FYP\BBox_Split\train"
annotation_csv = r"C:\Users\Travi\Downloads\BBox_Durian.csv"
output_image_dir = r"D:\Suvan's Projects\FYP\BBox_Split\train"
output_csv = r"D:\Suvan's Projects\FYP\Bbox_Augmentations.csv"

os.makedirs(output_image_dir, exist_ok=True)

# === Load Annotations ===
df = pd.read_csv(annotation_csv)

# === Convert bbox format (x_min, y_min, w, h) to Pascal VOC ===
def to_pascal_voc(x, y, w, h):
    return [x, y, x + w, y + h]

def from_pascal_voc(x_min, y_min, x_max, y_max):
    return [x_min, y_min, x_max - x_min, y_max - y_min]

# === Define Augmentations ===
# Unified aggressive augmentation for object detection
detection_augmentation = A.Compose([
    A.OneOf([
        A.HorizontalFlip(p=1),
        A.VerticalFlip(p=1),
        A.Affine(scale=(0.9, 1.1), rotate=(-45, 45), translate_percent={"x": (-0.15, 0.15), "y": (-0.15, 0.15)}, border_mode=cv2.BORDER_REFLECT_101, p=1),
        A.Perspective(scale=(0.08, 0.15), keep_size=True, p=1),
        A.RandomSizedBBoxSafeCrop(height = 640, width = 640, erosion_rate=0.2, p=1),
        A.RandomResizedCrop(size = (640,640), scale=(0.5, 0.85), ratio=(0.85, 1.15), p=1),
    ], p=1),

    A.OneOf([
        A.RandomBrightnessContrast(brightness_limit=0.25, contrast_limit=0.25, p=1),
        A.PlanckianJitter(mode='cied', p=1),
        A.Downscale(scale_range=(0.6, 0.75), p=1),
        A.CLAHE(clip_limit=2, tile_grid_size=(8, 8), p=1),
        A.HueSaturationValue(hue_shift_limit=15, sat_shift_limit=20, val_shift_limit=15, p=1),
        A.MotionBlur(blur_limit=7, p=1),
        A.Sharpen(p=1, alpha=(0.3, 0.7), lightness=(0.5, 1)),
    ], p=1),
],
    bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels'], min_visibility=0.1)
)

# === Augmentation Loop ===
augmented_data = []

# Process original data and add both original annotations and augmentations
for idx, row in tqdm(df.iterrows(), total=len(df)):
    filename = row['image_name']
    cls = row['label_name']
    x_min, y_min, w, h = row[['bbox_x', 'bbox_y', 'bbox_width', 'bbox_height']]

    # Process image for train only
    if os.path.exists(os.path.join(image_dir, filename)):
        image_path = os.path.join(image_dir, filename)
        image = cv2.imread(image_path)
        if image is None:
            continue

        height, width = image.shape[:2]
        bbox = to_pascal_voc(x_min, y_min, w, h)

        # Append original annotation (for all images including train, val, test)
        augmented_data.append([filename, cls, x_min, y_min, w, h])

        # Augmentation for training data (no need for 'train' check anymore)
        for i in range(4):  # Create 4 augmentations per image
            transformed = detection_augmentation(
                image=image,
                bboxes=[bbox],
                class_labels=[cls]
            )

            if not transformed['bboxes']:
                continue  # Skip if bbox got removed

            aug_img = transformed['image']
            aug_bbox = transformed['bboxes'][0]

            # Convert bbox back to (x_min, y_min, w, h)
            aug_xmin, aug_ymin, aug_w, aug_h = from_pascal_voc(*aug_bbox)

            new_filename = f"{os.path.splitext(filename)[0]}_aug{i}.jpg"
            cv2.imwrite(os.path.join(output_image_dir, new_filename), aug_img)
            augmented_data.append([new_filename, cls, aug_xmin, aug_ymin, aug_w, aug_h])

    else:
        # Append original annotation for val/test data directly
        augmented_data.append([filename, cls, x_min, y_min, w, h])

# === Save New CSV ===
aug_df = pd.DataFrame(augmented_data, columns=['filename', 'class', 'x_min', 'y_min', 'width', 'height'])
aug_df.to_csv(output_csv, index=False)

print("Augmentation completed and CSV updated!")


100%|██████████| 418/418 [00:05<00:00, 75.12it/s]

Augmentation completed and CSV updated!


## Converting CSV Annotations to YOLO text files

In [ ]:
import os
import pandas as pd
import cv2

# === Paths ===
image_root_dir = r"D:\Suvan's Projects\FYP\BBox_Split"
annotation_csv = r"D:\Suvan's Projects\FYP\Bbox_Augmentations.csv"
class_id = 0  # Single class: Leaf

# === Load CSV ===
df = pd.read_csv(annotation_csv)

# === Define YOLO conversion function ===
def convert_to_yolo(x_min, y_min, width, height, img_width, img_height):
    x_center = (x_min + width / 2) / img_width
    y_center = (y_min + height / 2) / img_height
    norm_width = width / img_width
    norm_height = height / img_height
    return x_center, y_center, norm_width, norm_height

# === Process Each Row ===
for idx, row in df.iterrows():
    filename = row['filename']
    found = False

    # Search across subfolders
    for split in ['train', 'val', 'test']:
        image_path = os.path.join(image_root_dir, split, filename)
        if os.path.exists(image_path):
            found = True
            img = cv2.imread(image_path)
            if img is None:
                print(f"⚠️ Could not read image: {image_path}")
                continue

            img_height, img_width = img.shape[:2]

            # Get and convert bbox
            x_min, y_min, width, height = row[['x_min', 'y_min', 'width', 'height']]
            x_center, y_center, norm_w, norm_h = convert_to_yolo(x_min, y_min, width, height, img_width, img_height)

            # Save YOLO-format .txt in same split directory
            txt_filename = os.path.splitext(filename)[0] + ".txt"
            txt_path = os.path.join(image_root_dir, split, txt_filename)

            with open(txt_path, 'w') as f:
                f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {norm_w:.6f} {norm_h:.6f}\n")
            break

    if not found:
        print(f"❌ Image not found in any split: {filename}")

print("✅ YOLO annotation conversion complete.")


✅ YOLO annotation conversion complete.


## Splitting Labels & Images for best practise

In [ ]:
import os
import shutil

# === Paths ===
base_dir = r"D:\Suvan's Projects\FYP\BBox_Split"
images_dir = os.path.join(base_dir, "images")
labels_dir = os.path.join(base_dir, "labels")

splits = ["train", "val", "test"]

for split in splits:
    image_split_dir = os.path.join(base_dir, split)
    target_image_dir = os.path.join(images_dir, split)
    target_label_dir = os.path.join(labels_dir, split)

    os.makedirs(target_image_dir, exist_ok=True)
    os.makedirs(target_label_dir, exist_ok=True)

    for file in os.listdir(image_split_dir):
        full_path = os.path.join(image_split_dir, file)

        if file.lower().endswith(".jpg") or file.lower().endswith(".png"):
            shutil.move(full_path, os.path.join(target_image_dir, file))

        elif file.lower().endswith(".txt"):
            shutil.move(full_path, os.path.join(target_label_dir, file))

print("✅ Annotation files moved to `labels/` and images organized in `images/`.")


✅ Annotation files moved to `labels/` and images organized in `images/`.


## Creating data.yaml package

In [ ]:
import yaml

data_yaml = {
    'train': 'D:/Suvan\'s Projects/FYP/BBox_Split/images/train',
    'val': 'D:/Suvan\'s Projects/FYP/BBox_Split/images/val',
    'test': 'D:/Suvan\'s Projects/FYP/BBox_Split/images/test',
    'nc': 1,
    'names': ['leaf']
}

with open(r"D:\Suvan's Projects\FYP\BBox_Split\data.yaml", 'w') as f:
    yaml.dump(data_yaml, f)

print("✅ data.yaml created successfully!")

✅ data.yaml created successfully!


## Building the YOLO model

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8s.pt")
model.train(
    data=r"D:/Suvan's Projects/FYP/BBox_Split/data.yaml",
    epochs=30,
    imgsz=640,
    batch=16,
    patience=10,
    verbose = True,
    name="leaf-detector"
)

Ultralytics 8.3.110  Python-3.12.8 torch-2.6.0+cpu CPU (AMD Ryzen 7 7435HS)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=D:/Suvan's Projects/FYP/BBox_Split/data.yaml, epochs=30, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=leaf-detector2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxe

train: Scanning D:\Suvan's Projects\FYP\BBox_Split\labels\train.cache... 1455 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1455/1455 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access  (ping: 0.10.0 ms, read: 667.7125.5 MB/s, size: 65.2 KB)


val: Scanning D:\Suvan's Projects\FYP\BBox_Split\labels\val.cache... 64 images, 0 backgrounds, 0 corrupt: 100%|██████████| 64/64 [00:00<?, ?it/s]

Plotting labels to runs\detect\leaf-detector2\labels.jpg... 


...

                   all         64         64      0.999      0.984      0.995      0.936



30 epochs completed in 5.686 hours.
Optimizer stripped from runs\detect\leaf-detector2\weights\last.pt, 22.5MB
Optimizer stripped from runs\detect\leaf-detector2\weights\best.pt, 22.5MB

Validating runs\detect\leaf-detector2\weights\best.pt...
Ultralytics 8.3.110  Python-3.12.8 torch-2.6.0+cpu CPU (AMD Ryzen 7 7435HS)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:10<00:00,  5.02s/it]


                   all         64         64      0.998      0.984      0.995      0.942
Speed: 1.6ms preprocess, 148.0ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs\detect\leaf-detector2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001C95E4A7200>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

## Metrics Visualization from YOLO model training/validation

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

csv_path = "runs/detect/leaf-detector2/results.csv"  # Adjust this path
save_dir = r"C:\Users\Travi\PycharmProjects\FYP\YOLO_Visualizations"

df = pd.read_csv(csv_path)

print("Columns in CSV:\n", df.columns.tolist())

metrics_to_plot = {
    'Train Box Loss': 'train/box_loss',
    'Train Class Loss': 'train/cls_loss',
    'Train DFL Loss': 'train/dfl_loss',
    'Validation Box Loss': 'val/box_loss',
    'Validation Class Loss': 'val/cls_loss',
    'Validation DFL Loss': 'val/dfl_loss',
    'Precision': 'metrics/precision(B)',
    'Recall': 'metrics/recall(B)',
    'mAP@0.5': 'metrics/mAP50(B)',
    'mAP@0.5:0.95': 'metrics/mAP50-95(B)'
}

sns.set(style='whitegrid')
for title, column in metrics_to_plot.items():
    if column not in df.columns:
        print(f"Warning: {column} not found in CSV. Skipping...")
        continue

    plt.figure(figsize=(10, 5))
    plt.plot(df.index + 1, df[column], marker='o')
    plt.title(f"{title} over Epochs")
    plt.xlabel("Epoch")
    plt.ylabel(title)
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, f"{title.replace(' ', '_')}.png"), dpi=300)
    plt.close()

print("Metrics visualized and saved to: {save_dir}")

Columns in CSV:
 ['epoch', 'time', 'train/box_loss', 'train/cls_loss', 'train/dfl_loss', 'metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'val/box_loss', 'val/cls_loss', 'val/dfl_loss', 'lr/pg0', 'lr/pg1', 'lr/pg2']
Metrics visualized and saved to: {save_dir}


## Metric Visualization from Test Set Evaluation

In [ ]:
from ultralytics import YOLO

model = YOLO("runs/detect/leaf-detector2/weights/best.pt")

metrics = model.val(
    data=r"D:/Suvan's Projects/FYP/BBox_Split/data.yaml",
    split='test',
    imgsz=640,
    batch=16
)

print(metrics)

Ultralytics 8.3.110  Python-3.12.8 torch-2.6.0+cpu CPU (AMD Ryzen 7 7435HS)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access  (ping: 0.20.1 ms, read: 113.415.2 MB/s, size: 64.7 KB)


val: Scanning D:\Suvan's Projects\FYP\BBox_Split\labels\test... 63 images, 0 backgrounds, 0 corrupt: 100%|██████████| 63/63 [00:00<00:00, 1539.58it/s]

val: New cache created: D:\Suvan's Projects\FYP\BBox_Split\labels\test.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:10<00:00,  2.73s/it]


                   all         63         63      0.999          1      0.995      0.931
Speed: 1.4ms preprocess, 161.3ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs\detect\val
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001BAB9FBB470>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.

## Image Visualization from Test Set Evaluation

In [ ]:
results = model.predict(
    source=r"D:/Suvan's Projects/FYP/BBox_Split/images/test",  # Test image directory
    imgsz=640,
    conf=0.25,
    save=True,
    save_txt=True,
    project="runs/test_inference",  # Save directory
    name="leaf_test"
)


image 1/63 D:\Suvan's Projects\FYP\BBox_Split\images\test\Algal_Leaf_Spot_14.jpg: 640x640 1 leaf, 255.3ms
image 2/63 D:\Suvan's Projects\FYP\BBox_Split\images\test\Algal_Leaf_Spot_15.jpg: 640x640 1 leaf, 155.6ms
image 3/63 D:\Suvan's Projects\FYP\BBox_Split\images\test\Algal_Leaf_Spot_18.jpg: 640x640 1 leaf, 157.5ms
image 4/63 D:\Suvan's Projects\FYP\BBox_Split\images\test\Algal_Leaf_Spot_20.jpg: 640x640 1 leaf, 158.9ms
image 5/63 D:\Suvan's Projects\FYP\BBox_Split\images\test\Algal_Leaf_Spot_24.jpg: 640x640 1 leaf, 156.2ms
image 6/63 D:\Suvan's Projects\FYP\BBox_Split\images\test\Algal_Leaf_Spot_28.jpg: 640x640 1 leaf, 162.9ms
image 7/63 D:\Suvan's Projects\FYP\BBox_Split\images\test\Algal_Leaf_Spot_40.jpg: 640x640 1 leaf, 157.4ms
image 8/63 D:\Suvan's Projects\FYP\BBox_Split\images\test\Algal_Leaf_Spot_42.jpg: 640x640 1 leaf, 161.3ms
image 9/63 D:\Suvan's Projects\FYP\BBox_Split\images\test\Algal_Leaf_Spot_47.jpg: 640x640 1 leaf, 160.5ms
image 10/63 D:\Suvan's Projects\FYP\BBox_Spli

## Testing Model with Real Life Data


In [ ]:
from ultralytics import YOLO
import cv2
import os

model = YOLO('runs/detect/leaf-detector2/weights/best.pt')
image_path = r"C:\Users\Travi\Downloads\WhatsApp Image 2025-04-10 at 11.39.27 AM (1).jpeg"

image = cv2.imread(image_path)

# Running the tests
results = model.predict(source=image_path, conf=0.2, save=False, verbose=False)

# Get results from the first image (only one image used)
boxes = results[0].boxes

# Extract all bounding boxes and compute their areas
if boxes is not None and len(boxes) > 0:
    largest_box = None
    max_area = 0

    for box in boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        area = (x2 - x1) * (y2 - y1)

        if area > max_area:
            max_area = area
            largest_box = (x1, y1, x2, y2)

    if largest_box:
        x1, y1, x2, y2 = largest_box

        # Draw the bounding box on the image
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box, thickness 2

        label = f"Confidence: {box.conf[0]:.2f}"
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Save the image with the bounding box
        save_path = 'output_with_largest_bbox1.jpg'  # Change this to your desired output path
        cv2.imwrite(save_path, image)

        print(f"[INFO] Image saved at: {save_path}")
    else:
        print("[WARNING] No valid bounding boxes found.")
else:
    print("[WARNING] No objects detected.")


[INFO] Image saved at: output_with_largest_bbox1.jpg


## Converting YOLOv8 model to TFLite format

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.8/978.8 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 36.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [4]:
from ultralytics import YOLO

model = YOLO('/content/drive/MyDrive/Colab Notebooks/YOLOv8_best.pt')
model.export(format = 'tflite')

Ultralytics 8.3.111 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.00GHz)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/Colab Notebooks/YOLOv8_best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (21.5 MB)
requirements: Ultralytics requirements ['sng4onnx>=1.0.1', 'onnx_graphsurgeon>=0.3.26', 'ai-edge-litert>=1.2.0', 'onnx>=1.12.0', 'onnx2tf>=1.26.3', 'onnxslim>=0.1.31', 'tflite_support', 'onnxruntime'] not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.7/147.7 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 294.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 182.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 319.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.

100%|██████████| 1.11M/1.11M [00:00<00:00, 60.0MB/s]
Unzipping calibration_image_sample_data_20x128x128x3_float32.npy.zip to /content/calibration_image_sample_data_20x128x128x3_float32.npy...: 100%|██████████| 1/1 [00:00<00:00, 49.96file/s]



ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.50...
ONNX: export success ✅ 2.4s, saved as '/content/drive/MyDrive/Colab Notebooks/YOLOv8_best.onnx' (42.8 MB)
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.27.2...
TensorFlow SavedModel: export success ✅ 124.2s, saved as '/content/drive/MyDrive/Colab Notebooks/YOLOv8_best_saved_model' (107.5 MB)

TensorFlow Lite: starting export with tensorflow 2.18.0...
TensorFlow Lite: export success ✅ 0.0s, saved as '/content/drive/MyDrive/Colab Notebooks/YOLOv8_best_saved_model/YOLOv8_best_float32.tflite' (42.7 MB)

Export complete (126.8s)
Results saved to /content/drive/MyDrive/Colab Notebooks
Predict:         yolo predict task=detect model=/content/drive/MyDrive/Colab Notebooks/YOLOv8_best_saved_model/YOLOv8_best_float32.tflite imgsz=640  
Validate:        yolo val task=detect model=/content/drive/MyDrive/Colab Notebooks/YOLOv8_best_saved_model/YOLOv8_best_float32.tflite imgsz=640 data=D:/Suvan'

'/content/drive/MyDrive/Colab Notebooks/YOLOv8_best_saved_model/YOLOv8_best_float32.tflite'